In [ ]:
import pandas as pd
import requests

# Método para obtener la información del juego
def obtener_tags_juego(app_id):
    url = f"http://store.steampowered.com/api/appdetails?appids={app_id}"
    try:
        respuesta = requests.get(url, timeout=10)  # Timeout de 10 segundos
        respuesta.raise_for_status()  # Lanzar una excepción para errores HTTP
        detalles_juego = respuesta.json().get(str(app_id), {})
        if 'success' in detalles_juego and detalles_juego['success'] is False:
            print(f"No se pudo obtener información para el juego con ID {app_id}.")
            return [], []
        if 'data' in detalles_juego:
            generos = [genero['description'] for genero in detalles_juego['data'].get('genres', [])]
            categorias = [categoria['description'] for categoria in detalles_juego['data'].get('categories', [])]
            return generos, categorias
    except requests.exceptions.RequestException as e:
        print(f"Error al realizar la solicitud para el juego con ID {app_id}: {e}")

    return [], []

# Leer el archivo CSV completo
df_csv = pd.read_csv("/content/sample_data/respuestasA (2400-158113).csv")

# Crear columnas para Géneros y Categorías
df_csv['Géneros'] = ""
df_csv['Categorías'] = ""

# Contador para el número de ejecuciones
num_ejecuciones = 0

# Iterar sobre la columna "ID del juego" y obtener las tags para cada juego
for index, row in df_csv.iterrows():
    num_ejecuciones += 1
    app_id = row['ID del juego']
    generos, categorias = obtener_tags_juego(app_id)
    if not generos and not categorias:
        print(f"No se obtuvieron tags para el juego con ID {app_id}.")
    else:
        df_csv.at[index, 'Géneros'] = ', '.join(generos)
        df_csv.at[index, 'Categorías'] = ', '.join(categorias)
        # Guardar el DataFrame actualizado con las nuevas columnas después de cada iteración
        df_csv.to_csv(f"/content/sample_data/respuestasA_con_tags_{num_ejecuciones}.csv", index=False)

print(f"Se han procesado {num_ejecuciones} juegos.")
